In [42]:
Выберите набор данных (датасет) для решения задачи классификации или регресии.

В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.

С использованием метода train_test_split разделите выборку на обучающую и тестовую.

Обучите следующие ансамблевые модели:

одну из моделей группы бэггинга (бэггинг или случайный лес или сверхслучайные деревья);
одну из моделей группы бустинга;
одну из моделей группы стекинга.
(+1 балл на экзамене) Дополнительно к указанным моделям обучите еще две модели:

Модель многослойного персептрона. По желанию, вместо библиотеки scikit-learn возможно использование библиотек TensorFlow, PyTorch или других аналогичных библиотек.
Модель МГУА с использованием библиотеки - https://github.com/kvoyager/GmdhPy (или аналогичных библиотек). Найдите такие параметры запуска модели, при которых она будет по крайней мере не хуже, чем одна из предыдущих ансамблевых моделей.
Оцените качество моделей с помощью одной из подходящих для задачи метрик. Сравните качество полученных моделей.

SyntaxError: invalid syntax (<ipython-input-42-464c986505a5>, line 1)

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

titanic_df = pd.read_csv('titanic.csv')

# Предобработка данных
# Кодирование категориальных признаков
le = LabelEncoder()
titanic_df['Sex'] = le.fit_transform(titanic_df['Sex'])

# Заполнение пропусков
titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace=True)
titanic_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Fare'], axis=1, inplace=True)
# Выбор признаков
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']
X = titanic_df[features]
y = titanic_df['Survived']
print(y)
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=2)
print(y_train)
# Масштабирование признаков
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)


0      0
1      1
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64
317    0
156    1
251    0
82     0
358    0
      ..
299    0
22     1
72     1
15     1
168    1
Name: Survived, Length: 209, dtype: int64


In [49]:
X_train

,Pclass,Sex,Age,SibSp,Parch
317,2,1,19.00000,0,0
156,1,0,29.00000,0,0
251,3,1,20.00000,0,0
82,1,1,49.00000,0,0
358,3,1,30.27259,0,0
...,...,...,...,...,...
299,3,1,29.00000,0,0
22,1,0,30.27259,0,0
72,3,0,29.00000,0,0
15,2,0,24.00000,1,0


In [50]:
y_train

317    0
156    1
251    0
82     0
358    0
      ..
299    0
22     1
72     1
15     1
168    1
Name: Survived, Length: 209, dtype: int64

**Обучение**

In [51]:
# Случайный лес
rf_clf = RandomForestClassifier(n_estimators=1, random_state=45)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print('Random Forest Accuracy: ', accuracy_score(y_test, y_pred_rf))

# XGBoost
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print('XGBoost Accuracy: ', accuracy_score(y_test, y_pred_xgb))

# Стекинг
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
]
stacking_clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
stacking_clf.fit(X_train, y_train)
y_pred_stack = stacking_clf.predict(X_test)
print('Stacking Classifier Accuracy: ', accuracy_score(y_test, y_pred_stack))


Random Forest Accuracy:  0.9617224880382775
XGBoost Accuracy:  1.0
Stacking Classifier Accuracy:  1.0


**Оценка качества**

In [52]:
# Оценка качества моделей
from sklearn.metrics import accuracy_score, roc_auc_score

# Точность (Accuracy) и AUC-ROC для случайного леса
accuracy_rf = accuracy_score(y_test, y_pred_rf)
auc_roc_rf = roc_auc_score(y_test, rf_clf.predict_proba(X_test)[:, 1])
print(f'Random Forest Accuracy: {accuracy_rf:.4f}, AUC-ROC: {auc_roc_rf:.4f}')

# Точность (Accuracy) и AUC-ROC для XGBoost
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
auc_roc_xgb = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
print(f'XGBoost Accuracy: {accuracy_xgb:.4f}, AUC-ROC: {auc_roc_xgb:.4f}')

# Точность (Accuracy) и AUC-ROC для стекинга
accuracy_stack = accuracy_score(y_test, y_pred_stack)
# Для AUC-ROC расчета в StackingClassifier нужно использовать метод predict_proba()
# Генерация вероятностей для класса 1 (жив)
stack_prob = stacking_clf.predict_proba(X_test)[:, 1]
auc_roc_stack = roc_auc_score(y_test, stack_prob)
print(f'Stacking Classifier Accuracy: {accuracy_stack:.4f}, AUC-ROC: {auc_roc_stack:.4f}')


Random Forest Accuracy: 0.9617, AUC-ROC: 0.9695
XGBoost Accuracy: 1.0000, AUC-ROC: 1.0000
Stacking Classifier Accuracy: 1.0000, AUC-ROC: 1.0000
